<a href="https://colab.research.google.com/github/YvonneHoven/AI1/blob/master/lab4_6981046_6654185.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 4: Recurrent models

This lab gives you practice with embeddings (word vectors, in this case) and recurrent neural models in NLP. The first part focuses on embeddings as input to a recurrent model, and the second part focuses on embeddings derived from recurrent models, applying them to the task of word sense disambuiguation following the approach from the original paper by Peters et al.

Everybody's machine is different and the neural computations required for this lab are more demanding than in the other assignments in this course. For this reason, it is advisable to use Google CoLab which guarantees a minimum level of performance. It is also recommended to use GPU acceleration; on CoLab, it can be turned on via <code>Runtime>Change Runtime type>GPU</code>.

## Part 1 (45 points)

In the first part of lab 4, we will play with training a recurrent model for part of speech tagging. As an easy exercise, you will observe what happens when you plug in pretrained word embeddings into a neural NLP model and experiment with different sizes of training data.

If you use Google Colab, it may be easiest to place this notebook and <code>lstm_tutorial.py</code> in <code>/Colab Notebooks</code> directory of your Google Drive. Run the code in the cell just below to enable CoLab to access the files on Drive. This will produce a link to an obscure redirect page which misleadingly suggests that you are getting Google Drive Desktop installation; what happens instead you are allowing the notebook running on Colab access to Drive. Copy the code that appears after you allow Drive access and paste it below.

In [13]:
#RUN THIS CELL IF USING COLAB TO USE GOOGLE DRIVE FOR STORING lstm_tutorial.py AND/OR DATA FILES
from google.colab import drive 
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks


The neural network solutions in this lab rely on AllenNLP library version 0.9.0 (the other code of this lab assignment may work incorrectly with more recent versions); <code>overrides</code> is required for compatibility. Linguistic resources are from NLTK version 3.6.2 and might work incorrectly in other versions. Install these before proceeding; installation process may vary depending on your system. On CoLab, this can be done via the following command:

In [ ]:
#IF USING COLAB, INSTALL allennlp AND nltk AS FOLLOWS
!pip install -U allennlp==0.9.0 overrides==3.1.0 nltk==3.6.2

**Before you start**,  import required modules:

In [25]:
import random
import nltk
import allennlp

If you run this for the first time, you may need to download various data using NLTK:

In [26]:
nltk.download('brown')
nltk.download('semcor')
nltk.download('wordnet')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package semcor to /root/nltk_data...
[nltk_data]   Package semcor is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Exercise 1: prepare the data (5 points)

Linguistic data come in a variety of formats. You already had a chance to play with POS-annotated corpus data in Lab 1.

In the first exercise, you will access POS-annotated data in one format (NLTK) and save it on the disk in a text format. Start with the tagged sentences from the Brown corpus, which can be retrieved as below:

In [17]:
nltk.corpus.brown.tagged_sents()

[[('The', 'AT'), ('Fulton', 'NP-TL'), ('County', 'NN-TL'), ('Grand', 'JJ-TL'), ('Jury', 'NN-TL'), ('said', 'VBD'), ('Friday', 'NR'), ('an', 'AT'), ('investigation', 'NN'), ('of', 'IN'), ("Atlanta's", 'NP$'), ('recent', 'JJ'), ('primary', 'NN'), ('election', 'NN'), ('produced', 'VBD'), ('``', '``'), ('no', 'AT'), ('evidence', 'NN'), ("''", "''"), ('that', 'CS'), ('any', 'DTI'), ('irregularities', 'NNS'), ('took', 'VBD'), ('place', 'NN'), ('.', '.')], [('The', 'AT'), ('jury', 'NN'), ('further', 'RBR'), ('said', 'VBD'), ('in', 'IN'), ('term-end', 'NN'), ('presentments', 'NNS'), ('that', 'CS'), ('the', 'AT'), ('City', 'NN-TL'), ('Executive', 'JJ-TL'), ('Committee', 'NN-TL'), (',', ','), ('which', 'WDT'), ('had', 'HVD'), ('over-all', 'JJ'), ('charge', 'NN'), ('of', 'IN'), ('the', 'AT'), ('election', 'NN'), (',', ','), ('``', '``'), ('deserves', 'VBZ'), ('the', 'AT'), ('praise', 'NN'), ('and', 'CC'), ('thanks', 'NNS'), ('of', 'IN'), ('the', 'AT'), ('City', 'NN-TL'), ('of', 'IN-TL'), ('Atlant

Now randomize the order of all sentences in the corpus using <code>random.shuffle()</code> function and select the first 50K sentences for training and the next 5K for validation.

In [18]:
#Write your code here
tagged_sents = list(nltk.corpus.brown.tagged_sents())
random.shuffle(tagged_sents)

training_brown = tagged_sents[0:50000]
validation_brown = tagged_sents[50000:55000]
testing_brown = tagged_sents[55000:]

Define a function for saving your datasets to a text file in the following format:
* one sentence per line
* tokens separated by spaces
* POS tag separated from the token by "###", for example <code>said###VBD</code>.

In [19]:
def write_posdata(sentences,outfile):
    #Write your code here
    file = open(outfile, "w") 

    for sent in sentences:
      sentence = ""
      
      for word in sent:
        token = word[0] + "###" + word[1]
        sentence += token + " "
      
      # Remove redundant space at the end of sentence
      sentence = sentence[:-1]
      file.write(sentence + "\n") 

    file.close() 

Now save your data partitions in different sizes. We will start with small data samples since training on a large dataset may be very slow depending on your machine. We won't use the full 50K sentence training set in this lab since this might take too long.

In [20]:
write_posdata(training_brown,"train_brown.txt")
write_posdata(validation_brown,"validation_brown.txt")
write_posdata(training_brown[:50],"train_brown_50.txt")
write_posdata(validation_brown[:50],"validation_brown_50.txt")
write_posdata(training_brown[:500],"train_brown_500.txt")
write_posdata(validation_brown[:500],"validation_brown_500.txt")
write_posdata(training_brown[:5000],"train_brown_5000.txt")
# The rule below was missing, so we've added it ourselves 
write_posdata(validation_brown[:5000], "validation_brown_5000.txt")

Congratulations, you have now saved the POS tagged data for model training purposes!

## Exercise 2: train neural POS tagger models (15 points)

We will now play with a neural model. You have installed <code>allennlp</code> which contains all necessary components for this and the training code for an LSTM model, which follows an old AllenNLP tutorial, is contained in <code>lstm_tutorial.py</code>. PLace the latter in the same directory as this notebook. Let us start by loading the model code and data, starting with a tiny sample for demonstration purposes. 

In [22]:
from lstm_tutorial import *

train_dataset_tiny = reader.read("train_brown_50.txt")
validation_dataset_tiny = reader.read("validation_brown_50.txt")

50it [00:00, 5975.30it/s]
50it [00:00, 5442.62it/s]


Fist of all we need to initialize the vocabulary and define an embedding (vector) for each token. We set the embedding size at 300, common in realistic applications. By default, the embeddings are initialized randomly and updated during trining (this can be changed but we start with a standard configuration). We also need to specify the <code>HIDDEN_DIM</code> parameter: the dimensionality of the hidden vector representations in the LSTM cell.

In [23]:
vocab_tiny = Vocabulary.from_instances(train_dataset_tiny + validation_dataset_tiny)

EMBEDDING_DIM = 300
HIDDEN_DIM = 20

token_embedding_tiny = Embedding(num_embeddings=vocab_tiny.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)

100%|██████████| 100/100 [00:00<00:00, 21485.01it/s]


Download the smallest pretrained word vector model from https://nlp.stanford.edu/projects/glove/, unzip it, and extract the relevant file <code>'glove.6B.300d.txt'</code> in your working directory. The size of the file is 1GB; if using Google Drive with Colab, make sure you have sufficient space. Downloading and uploading the file might take a few minutes. You can <b>either</b> upload the relevant file from your personal machine `or` use the code below directly from CoLab:

In [ ]:
#THIS CELL IS OPTIONAL, TO BE USED ON COLAB. YOU CAN USE wget AS BELOW OR ALTERNATIVELY UPLOAD GloVe EMBEDDINGS TO GOOGLE DRIVE FROM YOUR MACHINE
# download the file
!wget http://nlp.stanford.edu/data/glove.6B.zip
# unzip the file
!unzip -d . 'glove.6B.zip'
# remove useless contents
!rm 'glove.6B.200d.txt' 'glove.6B.100d.txt' 'glove.6B.50d.txt'

--2021-06-15 08:40:04--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-06-15 08:40:05--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-06-15 08:40:05--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

Initialize token embeddings with values from pretrained GloVe model:


In [ ]:
glove_token_embedding_tiny = Embedding.from_params(vocab=vocab_tiny,
                            params=Params({'pretrained_file':'glove.6B.300d.txt',
                                           'embedding_dim' : EMBEDDING_DIM}))

400000it [00:02, 151025.79it/s]


Now from embedding a single word with <code>token_embedding_tiny</code> we can proceed to mapping a word sequence into a sequence of vectors:

In [ ]:
word_embeddings_tiny = BasicTextFieldEmbedder({"tokens": token_embedding_tiny})

The following initializes parameters of an LSTM model using <code>word_embeddings_tiny</code> input encoding

In [ ]:
lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))

model_tiny = LstmTagger(word_embeddings_tiny, lstm, vocab_tiny)

Now define an LSTM model called <code>glove_model_tiny</code> that uses <code>glove_token_embedding_tiny</code>:

In [ ]:
#write your code here
glove_word_embeddings_tiny = BasicTextFieldEmbedder({"tokens": glove_token_embedding_tiny})
glove_model_tiny = LstmTagger(glove_word_embeddings_tiny, lstm, vocab_tiny)

Train the basic model for the tiny dataset. **Do not clear the output of this cell in the submitted version.**



In [ ]:
basic_trainer_tiny=initialize_trainer(model_tiny,vocab_tiny,train_dataset_tiny,validation_dataset_tiny,batch_size=50)
basic_trainer_tiny.train()

Streaminguitvoer ingekort tot de laatste 5000 regels.
    **kwargs).stdout
  File "/usr/lib/python3.7/subprocess.py", line 512, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader']' returned non-zero exit status 9.
accuracy: 0.1839, loss: 3.4445 ||: 100%|██████████| 1/1 [00:00<00:00, 10.01it/s]
accuracy: 0.1918, loss: 3.3818 ||: 100%|██████████| 1/1 [00:00<00:00, 40.42it/s]
unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/allennlp/common/util.py", line 378, in gpu_memory_mb
    encoding='utf-8')
  File "/usr/lib/python3.7/subprocess.py", line 411, in check_output
    **kwargs).stdout
  File "/usr/lib/python3.7/subprocess.py", line 512, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader']' returned non-zero e

{'best_epoch': 999,
 'best_validation_accuracy': 0.3674698795180723,
 'best_validation_loss': 2.8670880794525146,
 'epoch': 999,
 'peak_cpu_memory_MB': 828.284,
 'training_accuracy': 0.3604757548032937,
 'training_cpu_memory_MB': 828.284,
 'training_duration': '0:03:11.254995',
 'training_epochs': 999,
 'training_loss': 2.8681204319000244,
 'training_start_epoch': 0,
 'validation_accuracy': 0.3674698795180723,
 'validation_loss': 2.8670880794525146}

You have trained an LSTM POS tagger for the basic model. Now train the <code>glove_model_tiny</code>. **Do not clear the output of this cell in the submitted version.**

In [ ]:
#Write your code here
glove_trainer_tiny=initialize_trainer(glove_model_tiny,vocab_tiny,train_dataset_tiny,validation_dataset_tiny,batch_size=50)
glove_trainer_tiny.train()

Streaminguitvoer ingekort tot de laatste 5000 regels.
    **kwargs).stdout
  File "/usr/lib/python3.7/subprocess.py", line 512, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader']' returned non-zero exit status 9.
accuracy: 0.4831, loss: 2.3582 ||: 100%|██████████| 1/1 [00:00<00:00, 11.80it/s]
accuracy: 0.4438, loss: 2.6427 ||: 100%|██████████| 1/1 [00:00<00:00, 34.14it/s]
unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/allennlp/common/util.py", line 378, in gpu_memory_mb
    encoding='utf-8')
  File "/usr/lib/python3.7/subprocess.py", line 411, in check_output
    **kwargs).stdout
  File "/usr/lib/python3.7/subprocess.py", line 512, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader']' returned non-zero e

{'best_epoch': 999,
 'best_validation_accuracy': 0.49096385542168675,
 'best_validation_loss': 2.215188503265381,
 'epoch': 999,
 'peak_cpu_memory_MB': 828.284,
 'training_accuracy': 0.6312900274473925,
 'training_cpu_memory_MB': 828.284,
 'training_duration': '0:03:01.220491',
 'training_epochs': 999,
 'training_loss': 1.572365403175354,
 'training_start_epoch': 0,
 'validation_accuracy': 0.49096385542168675,
 'validation_loss': 2.215188503265381}

## Exercise 3: Explore training parameters (25 points)

Create separate models on the basis of bigger datasets: the 500 sentence training and 500 sentence validation and 5000 sentence training and 5000 sentence validation. Using the full training set (50K sentences) is optional (your machine might be too slow). Initialize and train the basic model on 500 sentence training and 500 sentence validation data. **Do not clear the output of this cell in the submitted version.**

In [ ]:
#train the basic model on 500 sentences

train_dataset_500 = reader.read("train_brown_500.txt")
validation_dataset_500 = reader.read("validation_brown_500.txt")

# Initialize the vocabulary and define an embedding 
vocab_500 = Vocabulary.from_instances(train_dataset_500 + validation_dataset_500)

EMBEDDING_DIM = 300
HIDDEN_DIM = 20

token_embedding_500 = Embedding(num_embeddings=vocab_500.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)

# Map word sequence into sequence of vectors
word_embeddings_500 = BasicTextFieldEmbedder({"tokens": token_embedding_500})

# Initialize parameters of the LSTM 
lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))
model_500 = LstmTagger(word_embeddings_500, lstm, vocab_500)


basic_trainer_500=initialize_trainer(model_500,vocab_500,train_dataset_500,validation_dataset_500,batch_size=500)
basic_trainer_500.train()


Streaminguitvoer ingekort tot de laatste 5000 regels.
    **kwargs).stdout
  File "/usr/lib/python3.7/subprocess.py", line 512, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader']' returned non-zero exit status 9.
accuracy: 0.1725, loss: 3.5713 ||: 100%|██████████| 1/1 [00:00<00:00,  1.30it/s]
accuracy: 0.1722, loss: 3.5612 ||: 100%|██████████| 1/1 [00:00<00:00,  2.87it/s]
unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/allennlp/common/util.py", line 378, in gpu_memory_mb
    encoding='utf-8')
  File "/usr/lib/python3.7/subprocess.py", line 411, in check_output
    **kwargs).stdout
  File "/usr/lib/python3.7/subprocess.py", line 512, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader']' returned non-zero e

{'best_epoch': 999,
 'best_validation_accuracy': 0.36049334757696416,
 'best_validation_loss': 3.083240509033203,
 'epoch': 999,
 'peak_cpu_memory_MB': 1177.016,
 'training_accuracy': 0.35021958941885406,
 'training_cpu_memory_MB': 1177.016,
 'training_duration': '0:18:39.759800',
 'training_epochs': 999,
 'training_loss': 3.0965018272399902,
 'training_start_epoch': 0,
 'validation_accuracy': 0.36049334757696416,
 'validation_loss': 3.083240509033203}

Now do the same training (500 sentence training and 500 sentence validation sets) with GloVE embeddings. **Do not clear the output of this cell in the submitted version.**

In [ ]:
#train the GloVE model on 500 sentences

train_dataset_500 = reader.read("train_brown_500.txt")
validation_dataset_500 = reader.read("validation_brown_500.txt")

# Initialize the vocabulary and define an embedding 
vocab_500 = Vocabulary.from_instances(train_dataset_500 + validation_dataset_500)

EMBEDDING_DIM = 300
HIDDEN_DIM = 20

glove_token_embedding_500 = Embedding.from_params(vocab=vocab_500,
                            params=Params({'pretrained_file':'glove.6B.300d.txt',
                                           'embedding_dim' : EMBEDDING_DIM}))

# Map word sequence into sequence of vectors
glove_word_embeddings_500 = BasicTextFieldEmbedder({"tokens": glove_token_embedding_500})

# Initialize parameters of the LSTM 
lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))
glove_model_500 = LstmTagger(glove_word_embeddings_500, lstm, vocab_500)


glove_basic_trainer_500=initialize_trainer(glove_model_500,vocab_500,train_dataset_500,validation_dataset_500,batch_size=500)
glove_basic_trainer_500.train()


Streaminguitvoer ingekort tot de laatste 5000 regels.
    **kwargs).stdout
  File "/usr/lib/python3.7/subprocess.py", line 512, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader']' returned non-zero exit status 9.
accuracy: 0.4306, loss: 2.7581 ||: 100%|██████████| 1/1 [00:00<00:00,  1.74it/s]
accuracy: 0.4349, loss: 2.7587 ||: 100%|██████████| 1/1 [00:00<00:00,  3.10it/s]
unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/allennlp/common/util.py", line 378, in gpu_memory_mb
    encoding='utf-8')
  File "/usr/lib/python3.7/subprocess.py", line 411, in check_output
    **kwargs).stdout
  File "/usr/lib/python3.7/subprocess.py", line 512, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader']' returned non-zero e

{'best_epoch': 999,
 'best_validation_accuracy': 0.5114110906089152,
 'best_validation_loss': 2.280092239379883,
 'epoch': 999,
 'peak_cpu_memory_MB': 1177.092,
 'training_accuracy': 0.5195587784700235,
 'training_cpu_memory_MB': 1177.092,
 'training_duration': '0:16:22.627695',
 'training_epochs': 999,
 'training_loss': 2.246457576751709,
 'training_start_epoch': 0,
 'validation_accuracy': 0.5114110906089152,
 'validation_loss': 2.280092239379883}

Use a bigger training set now with 5K sentence training and 5K sentence validation sets and random initial embeddings. **Do not clear the output of this cell in the submitted version.**

In [ ]:
#train the basic model on 5000 sentences

train_dataset_5000 = reader.read("train_brown_5000.txt")
validation_dataset_5000 = reader.read("validation_brown_5000.txt")

# Initialize the vocabulary and define an embedding 
vocab_5000 = Vocabulary.from_instances(train_dataset_5000 + validation_dataset_5000)

EMBEDDING_DIM = 300
HIDDEN_DIM = 20

token_embedding_5000 = Embedding(num_embeddings=vocab_5000.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)

# Map word sequence into sequence of vectors
word_embeddings_5000 = BasicTextFieldEmbedder({"tokens": token_embedding_5000})

# Initialize parameters of the LSTM 
lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))
model_5000 = LstmTagger(word_embeddings_5000, lstm, vocab_5000)


basic_trainer_5000=initialize_trainer(model_5000,vocab_5000,train_dataset_5000,validation_dataset_5000,batch_size=5000)
basic_trainer_5000.train()


Streaminguitvoer ingekort tot de laatste 5000 regels.
    **kwargs).stdout
  File "/usr/lib/python3.7/subprocess.py", line 512, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader']' returned non-zero exit status 9.
accuracy: 0.2332, loss: 3.5651 ||: 100%|██████████| 1/1 [00:13<00:00, 13.10s/it]
accuracy: 0.2348, loss: 3.5587 ||: 100%|██████████| 1/1 [00:03<00:00,  3.48s/it]
unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/allennlp/common/util.py", line 378, in gpu_memory_mb
    encoding='utf-8')
  File "/usr/lib/python3.7/subprocess.py", line 411, in check_output
    **kwargs).stdout
  File "/usr/lib/python3.7/subprocess.py", line 512, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader']' returned non-zero e

{'best_epoch': 999,
 'best_validation_accuracy': 0.3688804022913813,
 'best_validation_loss': 3.0137312412261963,
 'epoch': 999,
 'peak_cpu_memory_MB': 5194.424,
 'training_accuracy': 0.36861446910807005,
 'training_cpu_memory_MB': 5194.424,
 'training_duration': '4:41:43.568720',
 'training_epochs': 999,
 'training_loss': 3.0142531394958496,
 'training_start_epoch': 0,
 'validation_accuracy': 0.3688804022913813,
 'validation_loss': 3.0137312412261963}

Now do the same training (5K sentence training and 5K sentence validation sets) with GloVE embeddings. **Do not clear the output of this cell in the submitted version.**

In [ ]:
#train the GloVE model on 5000 sentences

train_dataset_5000 = reader.read("train_brown_5000.txt")
validation_dataset_5000 = reader.read("validation_brown_5000.txt")

# Initialize the vocabulary and define an embedding 
vocab_5000 = Vocabulary.from_instances(train_dataset_5000 + validation_dataset_5000)

EMBEDDING_DIM = 300
HIDDEN_DIM = 20

glove_token_embedding_5000 = Embedding.from_params(vocab=vocab_5000,
                            params=Params({'pretrained_file':'glove.6B.300d.txt',
                                           'embedding_dim' : EMBEDDING_DIM}))

# Map word sequence into sequence of vectors
glove_word_embeddings_5000 = BasicTextFieldEmbedder({"tokens": glove_token_embedding_5000})

# Initialize parameters of the LSTM 
lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))
glove_model_5000 = LstmTagger(glove_word_embeddings_5000, lstm, vocab_5000)


glove_basic_trainer_5000=initialize_trainer(glove_model_5000,vocab_5000,train_dataset_5000,validation_dataset_5000,batch_size=5000)
glove_basic_trainer_5000.train()

Streaminguitvoer ingekort tot de laatste 5000 regels.
  0%|          | 0/1 [00:00<?, ?it/s]
accuracy: 0.4101, loss: 2.8980 ||: 100%|██████████| 1/1 [00:14<00:00, 14.72s/it]

  0%|          | 0/1 [00:00<?, ?it/s]
accuracy: 0.4111, loss: 2.8980 ||: 100%|██████████| 1/1 [00:03<00:00,  3.13s/it]
unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/allennlp/common/util.py", line 378, in gpu_memory_mb
    encoding='utf-8')
  File "/usr/lib/python3.7/subprocess.py", line 411, in check_output
    **kwargs).stdout
  File "/usr/lib/python3.7/subprocess.py", line 512, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader']' returned non-zero exit status 9.

  0%|          | 0/1 [00:00<?, ?it/s]
accuracy: 0.4106, loss: 2.8963 ||: 100%|██████████| 1/1 [00:14<00:00, 14.75s/it]

  0%|          | 0/1 [00:00<?, ?it/s]
accuracy: 0.41

{'best_epoch': 999,
 'best_validation_accuracy': 0.47322266834461957,
 'best_validation_loss': 2.4920730590820312,
 'epoch': 999,
 'peak_cpu_memory_MB': 7287.312,
 'training_accuracy': 0.4731071678132412,
 'training_cpu_memory_MB': 7287.312,
 'training_duration': '5:02:22.872840',
 'training_epochs': 999,
 'training_loss': 2.4879066944122314,
 'training_start_epoch': 0,
 'validation_accuracy': 0.47322266834461957,
 'validation_loss': 2.4920730590820312}

For each trained model, record validation accuracy and training duration (they are returned along with other training stats after training a model) and accuracy on the training set. Fill in the numbers in the table below:

| model | validation accuracy | training accuracy | training duration|
|-------|---------------------|---------------|-------------------------------------------
| basic model on 50 sentences|0.3674698795180723|0.3604757548032937|0:03:11.254995|
| glove model on 50 sentences|0.49096385542168675|0.6312900274473925|0:03:01.220491|
| basic model on 500 sentences|0.36049334757696416|0.35021958941885406|0:18:39.759800|
| glove model on 500 sentences|0.5114110906089152|0.5195587784700235|0:16:22.627695|
| basic model on 5000 sentences|0.3688804022913813|0.36861446910807005|4:41:43.568720|
| glove model on 5000 sentences|0.47322266834461957|0.4731071678132412|5:02:22.872840|

**Question.** What do you conclude from these comparisons? when can it be especially beneficial to initialize a model with pretrained embeddings?

**Answer.** WRITE YOUR ANSWER HERE

It can be beneficial from early on, like 50 sentences (49% versus 36% and 63% versus 36% correct), with even a slightly faster run time. 
And with 500 sentences it is still beneficial; 51 vs 36 and 51 vs 35 and faster.
However, from 5K it shows only an increase of 11% (47 vs 36), while it takes a lot more time too!
So in general, when the data set is small, you can better use pretrained embeddings, but when the data set is big (5K or bigger), then you don't need it.

## Comment 
In this lab we used pretrained GloVe embeddings in a model for part of speech tagging. GloVe in its turn is also a neural word embedding model, but it had been trained on a completely different objective. GloVe vectors had been optimised on word cooccurrence matrix decomposition, i.e. on the task of predicting which words tend to occur with which other words. Part of speech certainly plays a role in determining statistical cooccurrence of words, but this role is indirect, and explicit part of speech information has not been used in training GloVe.

This makes our application an example of **transfer learning**, whereby a learned model trained on one objective (e.g. word cooccurrence) can benefit a different application (e.g. POS tagging), because some information is shared between them. 

## Part 2 - ELMo vectors (55 points)

> Indented block



In the second part of this lab we will reproduce the word sense disambiguation strategy that the authors of the ELMo vectors explored. The strategy consists in the following:

- create ELMo embeddings for all tokens in a sense-annotated corpus
- calculate mean sense vectors for each word sense in the training partition of the corpus
- for each sense-annotated token in the test partition of the corpus, assign it to the sense of the word to which its ELMo vector is the closest according to the cosine distance metric
- as a backup strategy, use the 1st sense of the word by default.

As a sense annotated corpus, we can use SemCor, conveniently available within NLTK. <code>semcor.sents()</code> iterates over all sentences represented as lists of tokens, while <code>semcor.tagged_sents()</code> iterates over the same sentences with additional annotation including WordNet lemma identifiers (lemmas in WordNet stand for a word taken in a specific sense).

In [27]:
from nltk.corpus import wordnet as wn  
from nltk.corpus import semcor
semcor.sents()
semcor.tagged_sents(tag="sem")

LookupError: ignored

## Exercise 1. Extract relevant data from SemCor (5 points)

Let's prepare SemCor data for the disambiguation task. Since this is just an educational exercise and we don't aim at replicating the full results, we can use only a subset of SemCor. Take the first 10K sentences of SemCor and split them randomly into 90% training and 10% testing partitions:

In [ ]:
import numpy as np

# Get first 10000 sentences of SemCor
sents = semcor.sents()[:10000]
tagged_sents = semcor.tagged_sents(tag="sem")[:10000]

# converting lists to np arrays
sents = np.array(sents)
tagged_sents = np.array(tagged_sents)

# Randomize sentences with seed=10
np.random.seed(seed=10)
np.random.shuffle(sents)
np.random.seed(seed=10)
np.random.shuffle(tagged_sents)

# Split sentences into train and test data
semcor_train = sents[:9000]
semcor_test = sents[9000:10000]

# Split tagged sentences into train and test data
semcor_train_tagged = tagged_sents[:9000]
semcor_test_tagged = tagged_sents[9000:10000]

Create a function that takes as input a sentence from SemCor and extracts a list which contains, for each token of the sentence, either the corresponding WordNet Lemma (e.g. <code>Lemma('friday.n.01.Friday')</code>) or <code>None</code>. <code>None</code> corresponds to tokens that are either 1) not annotated for word senses (e.g. articles); 2) are marked up as (part of) a named entity (e.g. "City of Atlanta" or placename "Fulton" annotated as  <code>Tree(Lemma('location.n.01.location'), [Tree('NE', ['Fulton'])])</code>). For example, <code>get_lemmas</code> for the first sentence of Semcor should return <code>[None,
 None,
 None,
 None,
 None,
 Lemma('state.v.01.say'),
 Lemma('friday.n.01.Friday'),
 None,
 Lemma('probe.n.01.investigation'),
 None,
 Lemma('atlanta.n.01.Atlanta'),
 None,
 Lemma('late.s.03.recent'),
 Lemma('primary.n.01.primary_election'),
 Lemma('primary.n.01.primary_election'),
 Lemma('produce.v.04.produce'),
 None,
 None,
 Lemma('evidence.n.01.evidence'),
 None,
 None,
 None,
 Lemma('abnormality.n.04.irregularity'),
 Lemma('happen.v.01.take_place'),
 Lemma('happen.v.01.take_place'),
 None]</code>

In [ ]:
def get_lemmas(semcor_sentence):
    Lemma_sentence = []

    for token in semcor_sentence:

      if type(token) is nltk.tree.Tree:
        if token.pos()[0][1] is 'NE':
          for pos in token.pos():
            Lemma_sentence.append(None)
        else:
          Lemma_sentence.append(token.pos()[0][1])
      else:
        Lemma_sentence.append(None)

    return Lemma_sentence

You are now able to extract word senses (instantiated by WordNet lemmas) from the corpus. The next step is to associate senses with ELMo vectors. Create a dictionary of contextualized token embeddings from the training corpus grouped by the WordNet sense:

In [ ]:
from collections import defaultdict

Train_embeddings=defaultdict(list)

Now let's create contextualized ELMo word embeddings for the tokens in this corpus. We can load the pretrained ELMo model and define a function <code>sentences_to_elmo()</code> that receives a list of tokenized sentences as input and produces their ELMo vectors.

In [ ]:
from allennlp.modules.elmo import Elmo, batch_to_ids

options_file = "https://allennlp.s3.amazonaws.com/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = "https://allennlp.s3.amazonaws.com/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"
elmo = Elmo(options_file, weight_file, 1, dropout=0)

def sentences_to_elmo(sentences):
    character_ids = batch_to_ids(sentences)
    embeddings = elmo(character_ids)
    return embeddings

Now you can process the corpus sentences and produce their ELMo vectors. It is recommended to pass the input to ELMo encoder in batches. A suggested batch size is 50 sentences. For example, the code below processes the first 50 sentences from the corpus:

In [ ]:
sentences=semcor.sents()[:50]

embeddings=sentences_to_elmo(sentences)

The <code>embeddings</code> that we obtained is a dictionary that contains a list of ELMo embeddings and a list of masks. The mask tells us which embeddings correspond to tokens in the original input sentences and which correspond to the padding (introduced to give all sentences in the batch the same length).
In principle all embeddings are stored in PyTorch tensors so that they can be used in bigger neural models, but we are not going to do it now.

In [ ]:
embeddings['elmo_representations'][0]

tensor([[[-6.4618e-03,  6.0215e-03, -3.5598e-01,  ..., -1.1715e-02,
           7.0427e-02, -4.1873e-01],
         [-3.7781e-01,  2.8141e-01, -2.5836e-01,  ..., -4.8547e-01,
           2.5508e-01,  3.6381e-02],
         [ 9.1191e-01,  1.1779e+00, -8.4833e-01,  ...,  9.8472e-01,
           3.3675e-01,  1.6172e-01],
         ...,
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[-6.4618e-03,  6.0215e-03, -3.5598e-01,  ..., -4.4876e-02,
           1.1313e-01, -9.9628e-02],
         [ 1.3721e-01, -2.0003e-01, -1.3074e-01,  ...,  5.9482e-01,
           9.3387e-01, -2.6757e-01],
         [ 1.7280e-01,  1.0801e+00, -5.4539e-01,  ...,  3.1966e-01,
          -5.6408e-01,  3.2461e-01],
         ...,
         [ 0.0000e+00,  0

We can check the size of the embeddings we got. It has three dimensions: 1) the number of sentences 2) the number of tokens (corresponds to the tokens in the longest original sentence of the batch; shorter ones were padded) and 3) the dimensionality of the Elmo vector (1024).

In [ ]:
embeddings['elmo_representations'][0].detach().size()

torch.Size([50, 59, 1024])

Another thing contained in the <code>embeddings</code> is the mask, a tensor encoding which token vectors correspond to original tokens and which are paddings. It has two dimensions, one corresponding to the sentences in the batch (50) and one corresponding to the token positions:

In [ ]:
print(embeddings['mask'].size())
embeddings['mask']

torch.Size([50, 59])


tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

## Exercise 2. Extract ELMo encoding of sentences using a mask (5 points)  

Now define a function <code>get_masked_vectors(embeddings)</code> that takes embeddings as input and returns a list of ELMo sentence encodings to which the mask has been applied.  The output should be a list of Torch tensors, where the padding vectors have been removed so each sentence is represented by an nx1024 tensor where n is sentence length.

In [ ]:
def get_masked_vectors(embeddings):
    masked_vectors = []
    
    elmo = embeddings['elmo_representations'][0].detach().numpy()
    mask = embeddings['mask'].detach().numpy()

    for i, elmo_sent in enumerate(elmo):
      masked_vectors.append(elmo_sent[mask[i]==1])


    return masked_vectors

## Exercise 3. Collect ELMo vectors from the training corpus (20 points)

Process the corpus updating your train word sense vectors in the dictionary. Iterate over the all the train sentences in the corpus, and retrieve for each lemma-annotated token (where lemma is not <code>None</code>) the corresponding ELMo vector. Store the ELMo sense embeddings that correspond to each lemma in the dictionary <code>Train_embeddings</code>. This step of processing the training corpus with ELMo is the most time consuming part of this assignment. However, it should not take forever. If this computation takes more than an hour, you may want to optimize your code or make sure you are using GPU acceleration. For the purposes of developing and debugging your solution, you may start by use a sample of 100 sentences, but then switch to the full 9K sentence training set. 

In [ ]:
import torch

training_sentences = semcor_train[:10]
tagged_training_sentences = semcor_train_tagged[:10]

elmo_embeddings = sentences_to_elmo(training_sentences)
elmo_vectors = get_masked_vectors(elmo_embeddings)

for i, sent in enumerate(training_sentences):
  lemmas = get_lemmas(tagged_training_sentences[i])
  
  for x, token in enumerate(lemmas):
    if token is not None:
      print(token)
      print(elmo_vectors[i][x])
      # Add Lemma (token) and ELMo vector to dictionary
      Train_embeddings[token] = elmo_vectors[i][x]

Lemma('date.n.03.engagement')
[-1.1461864   0.57018036  0.1331205  ... -0.36256114  0.34783375
 -0.04220193]
Lemma('be.v.01.be')
[-0.09777783 -0.46774364 -0.73126036 ...  0.8187541   0.07475835
 -0.77274525]
Lemma('fit.s.02.set')
[ 0.2952037   0.05366654  0.5537287  ... -0.09853639 -0.33031246
  0.12788442]
Lemma('large.a.01.big')
[-0.14213869 -0.33617106 -0.08957045 ...  0.04676433 -0.16669227
  0.33235827]
Lemma('theater.n.01.theater')
[ 0.11989184 -0.05964172  0.10566504 ... -0.00731318 -0.13821906
  0.00203437]
Lemma('ballet.n.01.ballet')
[-0.44267875 -0.5744685   0.08820968 ...  0.53966707  0.41141215
  0.19941159]
Lemma('booker.n.01.booker')
[ 0.0825282  -0.39727315  0.36174935 ...  0.4992206  -0.0166055
  0.01660493]
Lemma('extraordinary.s.03.extraordinary')
[ 0.00749363 -0.937676   -0.29632184 ...  0.75022566  0.04622305
 -0.15320732]
Lemma('see.v.05.consider')
[ 0.16792887 -0.07489969  0.40050748 ...  0.00747273  0.2022289
 -0.103677  ]
finest.s.00
[ 0.10796924  0.55878913 -0.

How many senses does your Train_embeddings contain? **Do not clear the output of this cell in your submission**

In [ ]:
print(len(Train_embeddings))

729


## Exercise 4. Vector averaging (5 points)

Your <code>Train_embeddings</code> now is a list of all vectors for a given word sense in the training corpus. For our purposes, we do not need the full list but the mean vector for each sense. For each sense in <code>Train_embeddings</code>, substitute the list by the average ELMo vector on the list. One efficient way to do this is to convert the list to a tensor via <code>stack</code> function and use Torch's <code>mean</code> function. Below is an example of how an average of two (random) vectors stored in a tensor can be computed in PyTorch: 

In [ ]:
 randtensor= torch.randn(2, 4)
 print("Tensor storing two 4-dimensional vectors:\n",randtensor)
 print("Average vector: \n",randtensor.mean(dim=0))

Tensor storing two 4-dimensional vectors:
 tensor([[-0.7794,  1.4825, -1.6353,  1.4017],
        [ 0.3750,  1.2442, -1.0922,  1.0940]])
Average vector: 
 tensor([-0.2022,  1.3633, -1.3637,  1.2478])


Now you are ready to update your <code>Train_embeddings</code> so that it maps lemmas not to lists but to averaged vectors.

In [ ]:
for lemma, elmo_vector in Train_embeddings.items():
  Train_embeddings[lemma] = np.mean(elmo_vector)

print(Train_embeddings)

defaultdict(<class 'list'>, {Lemma('date.n.03.engagement'): -0.021443222, Lemma('be.v.01.be'): -0.018214103, Lemma('fit.s.02.set'): -0.0077260714, Lemma('large.a.01.big'): 0.0024686474, Lemma('theater.n.01.theater'): -0.009725168, Lemma('ballet.n.01.ballet'): -0.0074679363, Lemma('booker.n.01.booker'): 0.012065239, Lemma('extraordinary.s.03.extraordinary'): -0.00060897134, Lemma('see.v.05.consider'): 0.008128398, 'finest.s.00': -0.008699112, Lemma('house.n.04.house'): -0.025540914, Lemma('kind.n.01.kind'): -0.014429934, Lemma('country.n.02.country'): -0.019074284, Lemma('naturally.r.01.of_course'): -0.0055543133, "n't.r.00": 0.006832787, Lemma('cry.v.02.weep'): -0.006640452, Lemma('capacity.n.03.capacity'): -0.008773955, Lemma('either.r.01.either'): -0.018624218, Lemma('hostess.n.02.hostess'): 0.033723265, Lemma('ask.v.01.enquire'): 0.03760258, Lemma('popularity.n.01.popularity'): -0.023291066, Lemma('folklore.n.01.folklore'): -0.018176563, Lemma('united_states.n.01.America'): -0.00793

## Exercise 5. Testing the sense vectors (20 points)

Test your sense embeddings on your test data, which is a subset of the SemCor corpus. Use the strategy outlined above, with 1st WordNet sense as a fallback: 

- rely on mean sense vectors for each word sense in the training partition of the corpus, as stored in <code>Train_embeddings</code>
- for each sense-annotated token <i>t</i> (e.g. the verb "run") in the test partition of the corpus, assign it to the sense of the word "Lemma('X.v.n.run')" to which ithe ELMo vector <i>t</i> is the closest according to the cosine distance metric
- as a backup strategy, use the 1st sense of the word (e.g. <code>Lemma('run.n.01.run')</code>) from WordNet. You can look it up using a built-in function from NLTK (e.g. <code>wn.lemmas('run')</code>).

Calculate WSD accuracy in percentage points on your test data. Report three numbers
- overall accuracy (proportion of times the ELMo method+WordNet backup results in the correct sense annotation)
- WordNet baseline accuracy: what if you always select the first WordNet sense, ignoring the ELMo embedding?
- accuracy of the ELMo method just for the instances in which ELMo strategy is applicable
- accuracy of the WordNet baseline just for the instances in which ELMo strategy is applicable

 **Do not delete the output of this cell in the submitted version**

In [ ]:
from torch.nn.functional import cosine_similarity

# Your code here
print("Overall accuracy:", accuracy) 
print("WordNet baseline:", baseline_accuracy)
print("Accuracy in cases where ELMo method is used", elmo_accuracy)
print("Accuracy of the baseline in cases where ELMo method is applicable", baseline_on_elmo_data_accuracy)


If you reached this point, you were able to evaluate ELMo as a model of contextual semantic similarity of word usages. The idea behind the vector averaging is that a word when used in the same sense should have similar vector representations, while usages in distinct senses should have different vector representations.

Analyze the numbers above. What do they tell you?

**WRITE YOUR ANALYSIS HERE**


## The end
Congratulations! this is the end of Lab 4.